In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [2]:
train_images = np.genfromtxt('data/MNIST/train.csv', delimiter=',', skip_header=1)
test_images = np.genfromtxt('data/MNIST/test.csv', delimiter=',', skip_header=1)

In [3]:
train_labels,train_images = train_images[:,0], train_images[:,1:] # label, features

In [4]:
(train_images.shape, train_labels.shape, test_images.shape)

((42000, 784), (42000,), (28000, 784))

In [5]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# Scale and convert the train images and add channels
train_images = np.asarray(train_images, dtype=np.float32) / 255
train_images = train_images.reshape((-1, 28, 28, 1)) #-1 will fit to any number of images

# Scale and convert the train images and add channels
test_images = np.asarray(test_images, dtype=np.float32) / 255
test_images = test_images.reshape((-1, 28, 28, 1))

In [6]:
(train_images.shape, train_labels.shape, test_images.shape)

((42000, 28, 28, 1), (42000,), (28000, 28, 28, 1))

In [7]:
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)

train_labels = train_labels.astype(np.float32)

In [8]:
model1 = tf.keras.Sequential()

model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model1.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                36928     
__________

In [9]:
model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model2.add(tf.keras.layers.Dropout(0.25))

model2.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                activation ='relu'))
model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model2.add(tf.keras.layers.Dropout(0.25))

model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(256, activation = "relu"))
model2.add(tf.keras.layers.Dropout(0.5))
model2.add(tf.keras.layers.Dense(10, activation = "softmax"))

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [10]:
model = model2

In [11]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001, epsilon=1e-08, decay=0.0)
# optimizer = tf.train.AdamOptimizer()  # performs worse than RMSProp

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [12]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [13]:
EPOCHS = 30

for epoch in range(EPOCHS):
  for (batch, (images, labels)) in enumerate(dataset):
    train_loss, train_accuracy = model.train_on_batch(images, labels)
    
    #if batch % 10 == 0: print(batch, train_accuracy)
  
  # Here you can gather any metrics or adjust your training parameters
  print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))

Epoch #1	 Loss: 0.001863	Accuracy: 1.000000
Epoch #2	 Loss: 0.071581	Accuracy: 0.937500
Epoch #3	 Loss: 0.000444	Accuracy: 1.000000
Epoch #4	 Loss: 0.292190	Accuracy: 0.937500
Epoch #5	 Loss: 0.000324	Accuracy: 1.000000
Epoch #6	 Loss: 0.096847	Accuracy: 0.937500
Epoch #7	 Loss: 0.151895	Accuracy: 0.937500
Epoch #8	 Loss: 0.082393	Accuracy: 0.937500
Epoch #9	 Loss: 0.000008	Accuracy: 1.000000
Epoch #10	 Loss: 0.030432	Accuracy: 1.000000
Epoch #11	 Loss: 0.164738	Accuracy: 0.937500
Epoch #12	 Loss: 0.000319	Accuracy: 1.000000
Epoch #13	 Loss: 0.005416	Accuracy: 1.000000
Epoch #14	 Loss: 0.000000	Accuracy: 1.000000
Epoch #15	 Loss: 0.000006	Accuracy: 1.000000
Epoch #16	 Loss: 0.000000	Accuracy: 1.000000
Epoch #17	 Loss: 0.151830	Accuracy: 0.937500
Epoch #18	 Loss: 0.000017	Accuracy: 1.000000
Epoch #19	 Loss: 0.000000	Accuracy: 1.000000
Epoch #20	 Loss: 0.000004	Accuracy: 1.000000
Epoch #21	 Loss: 0.000000	Accuracy: 1.000000
Epoch #22	 Loss: 0.000000	Accuracy: 1.000000
Epoch #23	 Loss: 0.

In [14]:
import os

os.makedirs('model', exist_ok=True)
os.makedirs('output', exist_ok=True)

In [15]:
tf.keras.models.save_model(
    model,
    "model/digit-recognizer-TF-Keras-CNN2",
    include_optimizer=False
)

In [16]:
md = tf.keras.models.load_model("model/digit-recognizer-TF-Keras-CNN2", compile=False)

In [17]:
prediction = md.predict(test_images)

In [18]:
label = tf.argmax(prediction,axis=1)
image_id = np.array(np.arange(1,TEST_SIZE+1))
result = np.column_stack((image_id, label))

In [19]:
np.savetxt("output/digit-recognizer-TF-Keras-CNN2.csv", result, fmt=('%d,%d'), delimiter=",", \
           header="ImageId,Label",comments='')

In [20]:
!kaggle competitions submit -c digit-recognizer -f output/digit-recognizer-TF-Keras-CNN2.csv -m "TF/Keras CNN2 with 30 epochs"

Successfully submitted to Digit Recognizer


__Kaggle Scores:__

Score: 0.99085 (30 epochs)
- model = tf.keras.Sequential()
- model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
- model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
- model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
- model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
- model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
- model.add(tf.keras.layers.Flatten())
- model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
- model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

Score: 0.98971 (30 epochs)

- model2 = tf.keras.Sequential()

- model2.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
- model2.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
- model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
- model2.add(tf.keras.layers.Dropout(0.25))
- model2.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
- model2.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
- model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
- model2.add(tf.keras.layers.Dropout(0.25))
- model2.add(tf.keras.layers.Flatten())
- model2.add(tf.keras.layers.Dense(256, activation = "relu"))
- model2.add(tf.keras.layers.Dropout(0.5))
- model2.add(tf.keras.layers.Dense(10, activation = "softmax"))